In [1]:
# pip install torch torchvision torchaudio


In [2]:
# pip install spire.doc transformers python-docx nltk sentence_transformers

In [2]:
# This code snippet is a Python script that uses the Spire.Doc library to extract text from a Word document. Here's a breakdown of what the code does:
from spire.doc import *
from spire.doc.common import *
from transformers import pipeline

# Создаем объект класса Document
doc = Document()

# Загружаем документ Word
doc.LoadFromFile("./instrukciya-po-ehkspluatacii-v8.docx")

# Создаем массив для хранения текста всех параграфов
text = []

# Перебираем все разделы в документе
for i in range(doc.Sections.Count):
    section = doc.Sections.get_Item(i)
    j = 0
    # Перебираем все дочерние объекты в разделе
    while j < section.Body.ChildObjects.Count:
        # Проверяем, является ли дочерний объект типа параграф
        if section.Body.ChildObjects[j].DocumentObjectType == DocumentObjectType.Paragraph:
            objItem = section.Body.ChildObjects[j]
            # Проверяем, является ли дочерний объект экземпляром класса Paragraph
            if isinstance(objItem, Paragraph):
                paraObj = Paragraph(objItem)
                # Добавляем текст параграфа в массив
                text.append(paraObj.Text)
                # Если текст параграфа пустой, удаляем объект из списка дочерних объектов раздела
                if len(paraObj.Text) == 0:
                    section.Body.ChildObjects.Remove(objItem)
                    j -= 1
        j += 1


In [4]:
from docx import Document

# Открываем документ Word
doc = Document("instrukciya-po-ehkspluatacii-v8.docx")

# Создаем список для хранения разделов текста
sections = []
current_section = []

# Перебираем все параграфы в документе
for paragraph in doc.paragraphs:
    # Проверяем стиль текущего параграфа
    if paragraph.style.name == "Heading 1":  # Здесь укажите имя стиля для ваших заголовков
        # Если текущий параграф является заголовком, добавляем предыдущий раздел текста в список секций
        if current_section:
            sections.append("\n".join(current_section))
            current_section = []
    # Добавляем текст параграфа к текущему разделу
    current_section.append(paragraph.text)

# Добавляем последний раздел текста в список секций
if current_section:
    sections.append("\n".join(current_section))

# # Выводим все разделы текста
# for i, section in enumerate(sections, start=1):
#     print(f"Section {i}:")
#     print(section)
#     print("-" * 50)


In [6]:
from docx import Document
import spacy


# Загрузка модели spacy для русского языка
nlp = spacy.load("ru_core_news_sm")


# Открываем документ Word
doc = Document("instrukciya-po-ehkspluatacii-v8.docx")

# Создаем список для хранения разделов текста
sections = []
current_section = []

# Перебираем все параграфы в документе
for paragraph in doc.paragraphs:
    # Проверяем стиль текущего параграфа
    if paragraph.style.name == "Heading 1":  # Здесь укажите имя стиля для ваших заголовков
        # Если текущий параграф является заголовком, добавляем предыдущий раздел текста в список секций
        if current_section:
            sections.append("\n".join(current_section))
            current_section = []
    # Добавляем текст параграфа к текущему разделу
    current_section.append(paragraph.text)

# Добавляем последний раздел текста в список секций
if current_section:
    sections.append("\n".join(current_section))

def lemmatize_text(text):
    doc = nlp(text)
    lemmas = [token.lemma_ for token in doc]
    return " ".join(lemmas)

lemmatized_sections = []

for section in sections:
    sentences = [sent.text for sent in nlp(section).sents]
    lemmatized_sentences = []
    
    for sentence in sentences:
        lemmatized_sentence = lemmatize_text(sentence)
        lemmatized_sentences.append(lemmatized_sentence)
    
    lemmatized_section = " ".join(lemmatized_sentences)
    lemmatized_sections.append(lemmatized_section)

# Выводим лемматизированные разделы текста
for i, section in enumerate(lemmatized_sections, start=1):
    print(f"Section {i}:")
    print(section)
    print("-" * 50)


Section 1:




 инструкция по эксплуатация 
 " 1с : предприятие 8 " 






 москва фирма " 1с " 2020 

 право тиражирования программных средств и документации 
 принадлежит фирме " 1с " 
 приобретать система " 1с : предприятие " , вы тем самым давать согласие не допускать копирование программа и документация без письменный разрешение фирма " 1с " 
 © ооо " 1с " , 2020 
 фирма " 1с " , москва , 123056 , а / я 64 . отдел продажа : селезневская ул. , 21 , телефон : ( 495 ) 737 - 92 - 57 , 
 факс : ( 495 ) 681 - 44 - 07 . 
 e - mail : 1c@1c.ru 
 url : www.1c.ru , www.v8.1c.ru 
 группа разработка программа – а. алексеев , а. безбородов , д. бескоровайнов , п. василец ,    а. виноградов ,    а. волков ,     и. гольштейн ,    е. горностаев , г. дамье , а. даровский , о. дерут , н. евграфов , б. евтифеев , д. зарецкий , д. ивашов ,    с. копиенко ,    н. корсаков ,     с. кравченко ,    в.    кудрявцев , п. кукушкин , а. лакутин , м. лейбович , г. леонтьев , а. лехан , а. макеев , а. медведев 

In [23]:
from sentence_transformers import SentenceTransformer, util
import torch

model = SentenceTransformer("all-MiniLM-L6-v2")

docs = section

document_embeddings = model.encode(docs)

query = "Что такое интерфейс"
query_embedding = model.encode(query)

query_embedding = model.encode(query, convert_to_tensor=True)
top_k = min(5, len(docs))
# We use cosine-similarity and torch.topk to find the highest 5 scores
cos_scores = util.cos_sim(query_embedding, document_embeddings)[0]
top_results = torch.topk(cos_scores, k=top_k print("\n\n======================\n\n")
print("Query:", query)
print("\nTop 5 most similar sentences in corpus:" for score, idx in zip(top_results[0], top_results[1]):
    print(corpus[idx], "(Score: {:.4f})".format(score))



SyntaxError: invalid syntax. Perhaps you forgot a comma? (962903610.py, line 17)

In [11]:
from sentence_transformers import SentenceTransformer, util
import torch

embedder = SentenceTransformer("all-MiniLM-L6-v2")

# Corpus with example sentences
corpus = lemmatized_sections
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

# Query sentences:
queries = [
    "Интерфейс системы",

]


# Проверка доступности CUDA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = min(5, len(corpus))
for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True, device=device)

    # We use cosine-similarity and torch.topk to find the highest 5 scores
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for score, idx in zip(top_results[0], top_results[1]):
        print(corpus[idx], "(Score: {:.4f})".format(score))

    """
    # Alternatively, we can also use util.semantic_search to perform cosine similarty + topk
    hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=5)
    hits = hits[0]      #Get the hits for the first query
    for hit in hits:
        print(corpus[hit['corpus_id']], "(Score: {:.4f})".format(hit['score']))
    """





Query: Интерфейс системы

Top 5 most similar sentences in corpus:
глава 15 . 	 режим обычный приложение 


 данная глава содержать информация по работа с конфигурациями , разработать специально для режим обычный приложение или создать на платформа версия 8.1 . 

 интерфейс 
 основный окно программа в режим " обычный приложение " значительно отличаться от режим управляемый приложение и в общий случай мочь иметь вид , как показать на рисунок низкий . 
 главный меню содержать команда работа с файл , редактирования , вызов форма объект конфигурация и т . д. 
 на панелях инструмент располагаться команда вызов стандартный и сервисный функция . Для переход к список и объект информационный база использоваться меню операция и кнопка панель инструмент , если определить на этап конфигурирование . 

 меню " операция " 
 меню операция использоваться для переход к список и объект информационный база , а также для вызов сервисный функция . При выбор тип объект в меню операция на экран отобразится

In [ ]:
from docx import Document
import spacy
from sentence_transformers import SentenceTransformer, util
import torch

# Загрузка модели spacy для русского языка
nlp = spacy.load("ru_core_news_sm")

# Открываем документ Word
doc = Document("Instructions1CEcology-Belov.docx")

# Создаем список для хранения разделов текста
sections = []
current_section = []

# Перебираем все параграфы в документе
for paragraph in doc.paragraphs:
    # Проверяем стиль текущего параграфа
    if paragraph.style.name == "Heading 1":  # Здесь укажите имя стиля для ваших заголовков
        # Если текущий параграф является заголовком, добавляем предыдущий раздел текста в список секций
        if current_section:
            sections.append("\n".join(current_section))
            current_section = []
    # Добавляем текст параграфа к текущему разделу
    current_section.append(paragraph.text)

# Добавляем последний раздел текста в список секций
if current_section:
    sections.append("\n".join(current_section))

def lemmatize_text(text):
    doc = nlp(text)
    lemmas = [token.lemma_ for token in doc]
    return " ".join(lemmas)

lemmatized_sections = []

for section in sections:
    sentences = [sent.text for sent in nlp(section).sents]
    lemmatized_sentences = []
    
    for sentence in sentences:
        lemmatized_sentence = lemmatize_text(sentence)
        lemmatized_sentences.append(lemmatized_sentence)
    
    lemmatized_section = " ".join(lemmatized_sentences)
    lemmatized_sections.append(lemmatized_section)

# Получаем эмбеддинги для предложений в каждой секции
embedder = SentenceTransformer("all-MiniLM-L6-v2")

# Проверка доступности CUDA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Создаем словарь для хранения предложений и их эмбеддингов по секциям
section_sentences = []
section_embeddings = []

for i, section in enumerate(sections):
    sentences = [sent.text for sent in nlp(section).sents]
    embeddings = embedder.encode(sentences, convert_to_tensor=True, device=device)
    section_sentences.append(sentences)
    section_embeddings.append(embeddings)

# Query sentences:
queries = [
    "Для получения доступа к ИС необходимо сформировать обращение на Портале самообслуживания:",
]

# Поиск наиболее похожих предложений для каждого запроса
top_k = 5
for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True, device=device)

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    results = []

    for section_idx, (sentences, embeddings) in enumerate(zip(section_sentences, section_embeddings)):
        cos_scores = util.cos_sim(query_embedding, embeddings)[0]
        top_results = torch.topk(cos_scores, k=min(top_k, len(sentences)))
        for score, idx in zip(top_results[0], top_results[1]):
            results.append((score.item(), section_idx, sentences[idx]))

    # Сортировка результатов по убыванию сходства
    results = sorted(results, key=lambda x: x[0], reverse=True)[:top_k]

    for score, section_idx, sentence in results:
        print(f"Section {section_idx + 1}: {sentence} (Score: {score:.4f})")






Query: Для получения доступа к ИС необходимо сформировать обращение на Портале самообслуживания:

Top 5 most similar sentences in corpus:
Section 4:  Основные элементы пользовательского интерфейса
Предприятие 
В разделе имеются документы для заведения основных данных по организациям и сотрудникам, а также нормативная документация.

 (Score: 0.8025)
Section 1: Инструкция предназначена для сотрудников, работающих в системе 1С Экология для ведения отчетности по всем направлениям защиты окружающей среды. (Score: 0.7766)
Section 4: Рисунок 9

Мероприятия
В разделе имеются документы для ведения деятельности по управлению и поставновке задач сотрудникам.

 (Score: 0.7751)
Section 4: Рисунок 8

Проверки
В разделе имеются документы для ведения деятельности по организации проверок на предприятии.

 (Score: 0.7718)
Section 4: Рисунок 5

Экологические показатели
В разделе имеются документы для отслеживания деятельности предприятия.

 (Score: 0.7628)


In [13]:
import torch
from transformers import GPT2Tokenizer, T5ForConditionalGeneration 
tokenizer = GPT2Tokenizer.from_pretrained('RussianNLP/FRED-T5-Summarizer',eos_token='</s>')
model = T5ForConditionalGeneration.from_pretrained('RussianNLP/FRED-T5-Summarizer')
# device='cuda'
# model.to(device)

input_text='<LM> О чем говорится в следующем тексте.\nДля получения доступа к ИС необходимо сформировать обращение на Портале самообслуживания: 1)Зайти на Портал самообслуживания по ссылке https://pso.enplus.group/; 2)Нажать на кнопку «Настройка прав доступа» в блоке «Быстрый доступ».'
input_ids=torch.tensor([tokenizer.encode(input_text)])
outputs=model.generate(input_ids,eos_token_id=tokenizer.eos_token_id,
                    num_beams=5,
                    min_new_tokens=17,
                    max_new_tokens=200,
                    do_sample=True,
                    no_repeat_ngram_size=4,
                    top_p=0.9)
print(tokenizer.decode(outputs[0][1:]))


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


KeyboardInterrupt: 